Before beginning Task 3, make sure to run the following cell to import all necessary packages. If you need any additional packages, add the import statement(s) to the cell below and re-run the cell before adding and running code that uses the additional packages.

In [ ]:
# Load all necessary packages
import numpy as np
import sklearn as skl
import six
import tensorflow as tf

# dataset
from aif360.datasets import AdultDataset

# metrics
from fklearn.metric_library import UnifiedMetricLibrary, classifier_quality_score

# models
from fklearn.scikit_learn_wrapper import LogisticRegression, KNeighborsClassifier, RandomForestClassifier, SVC
from aif360.algorithms.inprocessing import AdversarialDebiasing

# pre/post-processing algorithms
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, RejectOptionClassification

# Tutorial 3: AI Fairness 360

Lastly, we will show you how to use algorithms provided by AI Fairness 360 mitigate bias when training machine learning models. You will use the knowledge from this tutorial to complete Task 3, so please read thoroughly and execute the code cells in order.


## Step 1: Import the dataset

First we need to import the dataset we are going to use for training and testing.

Below we provide code that imports the Adult census dataset. **Note: a warning may pop up when you run this cell. As long as you don't see any errors in the code, it is fine to continue.**

In [ ]:
data_orig = AdultDataset()


## Step 2: Split the dataset into train and test data

Again, we want to split our dataset into train and test data as shown in the code below.

In [ ]:
data_orig_train, data_orig_test = data_orig.split([0.7], shuffle=False)


## Step 3: Set protected attributes

To use the bias mitigations features provided by AI Fairness 360, we need to set the privileged and unprivileged (protected) attributes. 

Below we provide code that stores the protected attributes (*race* is 0 for "Non-white", *sex* is 0 for "Female").

In [ ]:
unprivileged = [{'race': 0, 'sex': 0}]
privileged = [{'race': 1, 'sex': 1}]

## Step 4: Initialize model

Next, we initialize the model we want to evaluate. With AI Fairness 360, we can use any of the scikit-learn models or use the fair-aware Adversarial Debiasing model. 

Since initializing the Adversarial Debiasing model is slightly different from the scikit-learn models, we provide code in the next cell that initializes the Adversarial Debiasing model and a Logistic Regression model to show how the bias mitigation algorithms provided by AI Fairness 360 work.

Along with the code to execute for this tutorial, we also provide (commented) code that showcases how to initialize each of the models available for this task.

When using Adversarial Debiasing, we first need to create a TensorFlow session (as shown below).

**Note: when using the Adversarial Debiasing model, after the first initialization you activate a TensorFlow session. In the event you need to re-run a cell that created the Adversarial Debiasing model, make sure to add the following lines *before* the line `sess = tf.Session()`:**

`sess.close()`

`tf.reset_default_graph()`

In [ ]:
# model is populated with default values; modifying parameters is allowed but optional
model = LogisticRegression(penalty='l2', dual=False,tol=0.0001,C=1.0,
                       fit_intercept=True,intercept_scaling=1,class_weight=None,
                       random_state=None,solver='liblinear',max_iter=100, 
                       multi_class='warn',verbose=0,warm_start=False,
                       n_jobs=None)

#model = KNeighborsClassifier(n_neighbors=5,weights='uniform',algorithm='auto',
#                          leaf_size=30,p=2,metric='minkowski',metric_params=None,
#                          n_jobs=None)

#model = RandomForestClassifier(n_estimators='warn',criterion='gini',max_depth=None,
#                            min_samples_leaf=1,min_weight_fraction_leaf=0.0,
#                            min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, 
#                             random_state=None, verbose=0, warm_start=False, class_weight=None)

#model = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto_deprecated', coef0=0.0, shrinking=True, 
#          probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
#          max_iter=-1, decision_function_shape='ovr', random_state=None)

# If this is not your first time creating the Adversarial Debiasing model, to avoid future errors,
# uncomment the code below before running the code that initializing TensorFlow session and model:
# sess.close()
# tf.reset_default_graph()

#sess = tf.Session()
#model = AdversarialDebiasing(privileged_groups=privileged,
#                          unprivileged_groups=unprivileged,
#                          scope_name='debiased_classifier',
#                          debias=True,
#                          sess=sess)

## Step 5: Mitigate bias in dataset

When completing the next tasks, one way you can try to mitigate bias is by pre-processing the dataset. AI Fairness 360 provides algorithms for mitigating bias in the dataset.

Below we provide code that uses the Disparate Impact Remover pre-processing algorithm to show how to transform data for training your model. We also provide (commented) code that shows how to initialize each pre-processing algorithm available for use in this exercise. We also provide (commented) example code on how to use the pre-processor to transform the train data -- the same code is used to transform test data.

In [ ]:
# you can modify repair level (optional)
pre_alg = DisparateImpactRemover(repair_level=1.0)
# training data
pre_train_data = pre_alg.fit_transform(data_orig_train)
# test data
pre_test_data = pre_alg.fit_transform(data_orig_test)


# Reweighing
#pre_alg = Reweighing(unprivileged_groups=unprivileged, privileged_groups=privileged)
# train
#pre_alg = pre_alg.fit(data_orig_train)
#pre_train_data = pre_alg.transform(data_orig_train)
# test
#pre_alg = pre_alg.fit(data_orig_test)
#pre_test_data = pre_alg.transform(data_orig_test)


## Step 6: Train model 

Regardless of whether we pre-processed the dataset, we need to train our model to be able to evaluate it. Since we ran a pre-processing algorithm, we'll use our pre-processed dataset to train our model. If we hadn't pre-processed the data, or want to train the model with the original training data, we would pass in our original training dataset (`data_orig_train`).

Below we provide code that trains the Logistic Regression model using the pre-processed dataset.


In [ ]:
# train model with pre-processed data
model.fit(pre_train_data)

# train model with original data
# model.fit(data_orig_train)

## Step 7: Mitigate bias in trained model

Another way you can try to mitigate bias when completing the next tasks is to run a post-processing algorithm on the trained model. AI Fairness 360 also provides algorithms for mitigating bias in trained models.

Below we provide code that uses the Calibrated Equal Odds Post-processing algorithm to mitigate bias in our trained Logistic Regression model. We also provide (commented) code that shows how to initialize each of the post-processing algorithms available for use in this exercise. The steps required to use each post-processing algorithm is the same as shown in the code below.

In [ ]:
# process trained model
# Calibrated Equal Odds
post_alg = CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged,
                                         privileged_groups=privileged,
                                         cost_constraint='weighted',
                                         seed=None)


# Reject Option Classification 

# With this algorithm, you can specify "metric_name" with the metric you want to optimize for.
# The options are "Statistical parity difference", "Average odds difference", or "Equal opportunity difference"
# post_alg = RejectOptionClassification(unprivileged_groups=unprivileged,
#                                      privileged_groups=privileged,
#                                      low_class_thresh=0.01,
#                                      high_class_thresh=0.99,num_class_thresh=100, 
#                                      num_ROC_margin=50,metric_name='Statistical parity difference',
#                                      metric_ub=0.05, metric_lb=-0.05)


# test with pre-processed data
predictions = model.predict(pre_test_data)

# test with original data
# predictions = model.predict(data_orig_test)

# fit with post-processing model with pre-processed data
post_model = post_alg.fit(pre_test_data, predictions)

# fit with post-processing model with original data
# post_model = post_alg.fit(data_orig_test, predictions)



## Step 8: Evaluate the model

Now we're ready to evaluate our model. When completing the tasks that follow, you can evaluate any trained model (with or without pre-/post-processing) at any time.

Below we provide code to evaluate our post-processed Logistic Regression model on the pre-processed test dataset for performance, fairness, and overall quality. We also provided (commented) code that shows the various configurations you can evaluate.

In [ ]:
# test with original model/original data
# predictions = model.predict(data_orig_test)

# test with original model/pre-processed data
# predictions = model.predict(pre_test_data)

# test with post-processed model/original data
# predictions = post_model.predict(data_orig_test)

# test with post-processed model/pre-processed data
predictions = post_model.predict(pre_test_data)


# evaluate models based on pre-processed data
metric_lib = UnifiedMetricLibrary(pre_test_data, predictions,
                                 unprivileged_groups=unprivileged,
                                 privileged_groups=privileged)

# evaluate models based on original data
#metric_lib = UnifiedMetricLibrary(data_orig_test, predictions,
#                                 unprivileged_groups=unprivileged,
#                                 privileged_groups=privileged)


# accuracy (performance)
accuracy = metric_lib.accuracy_score()
print("Accuracy = " + str(accuracy))

# equal opportunity difference (fairness)
eq_opp_diff = metric_lib.equal_opportunity_difference()
print("Equal opportunity difference = " + str(eq_opp_diff))

# disaprate impact (fairness)
disp_impact = metric_lib.disparate_impact()
print("Disparate impact = " + str(disp_impact))

# statistical parity difference (fairness)
stat_parity_diff = metric_lib.statistical_parity_difference()
print("Statistical parity difference = " + str(stat_parity_diff))

# average odds difference (fairness)
avg_odds_diff = metric_lib.average_odds_difference()
print("Average odds difference = " + str(avg_odds_diff))


#overall quality for post-processed model
quality_score = classifier_quality_score(post_model, predictions, 
                                             unprivileged_groups=unprivileged, 
                                             privileged_groups=privileged)

#overall quality for original model
#quality_score = classifier_quality_score(model, predictions, 
#                                             unprivileged_groups=unprivileged, 
#                                             privileged_groups=privileged)

print("Overall quality = " + str(quality_score))

# Task 3: Model evaluation with AI Fairness 360

Now, use what you learned in the above tutorial to train and evaluate models for performance, fairness, and overall quality. You will use functionality provided by AI Fairness 360 to meet the following goals:

1. **Describe a model you believe will perform the best (e.g., have the highest accuracy score).** 

2. **Describe a model you believe will be the most fair, regardless of performance (e.g., minimizes the value of difference fairness metrics or maximizes disparate impact).** 

3. **Describe a model you believe will best balance both performance and fairness (e.g., have the highest classifier quality score).** 

Make sure you include any modifications to model hyper-parameters and any pre-/post-processing algorithms used. **As a reminder, there is no "absolute best" model for each of the above goals. You are expected to explore the space of model configurations available to find a model that best meets the above goals.**

**Keep in mind, training machine learning models is often a time intensive endeavor.** One way you can minimize time to finish the assignment is to minimize the times you have to, for example, train a given model to then evaluate it. You can do this by putting the code that initializes and trains your model(s) in its own separate cell and only execute this cell when needed.


## Submitting your response

Once you feel you've met the above goals, go to the Evaluating ML Models Exercise Response Form to enter your responses under the section labeled 'Task 3'.

If you accidentally closed your response form, check your email for the link to re-open it.

In [ ]:
# TODO : Use this cell to write code for completing task 3




Once you've completed this final task, make sure you're satisfied with your responses, complete the exercise feedback portion, and submit the form.